In [1]:
%run prophet_funcs_v2.py
#initialization script creates functions and adds libraries to environment

# Download Financial data from Yahoo Finance
The following code has two inputs that must be set to retrieve data from Yahoo Finance API, 'tickerStrings' and 'period'. The function 'FetchData_dict' retrieves the ticker data for the specified period and adds them to a data dictionary called 'df_dict'. 

The retrieved data consists of the following:
* **Date**
* **High**
* **Low**
* **Close**
* **Adj Close**
* **Volume**

note: yFinace is the primary library used to call the Yahoo Finance API.

In [4]:
'''Download Financial Data from Yahoo Finance'''

@ipyw.interact_manual (
tickerStrings= ipyw.widgets.SelectMultiple(
    options= ['SOL1-USD','BTC-USD','ETH-USD','1INCH-USD','ADA-USD'],
    value= ['SOL1-USD','BTC-USD','ETH-USD','1INCH-USD','ADA-USD']),
period= ['1d','5d','1mo','3mo','6mo','1y','2y','5y','10y','ytd','max']
)



def FetchData_dict(tickerStrings,period):
    global df_dict
    df_list = list()
    for ticker in tickerStrings:
        data = yf.download(ticker, group_by="Ticker", period = period)
        data['ticker'] = ticker  # add this column because the dataframe doesn't contain a column with the ticker
        df_list.append(data)
        df = pd.concat(df_list) # combine all dataframes into a single dataframe
        df_dict = {value: df[df['ticker'] == value].drop('ticker', axis=1) for value in df['ticker'].unique()}

interactive(children=(SelectMultiple(description='tickerStrings', index=(0, 1, 2, 3, 4), options=('SOL1-USD', …

# Plotting individual tickers
The following code uses ipython widgets to make it easy to view plots for each ticker that is downloaded. You can select the downloaded data you wish to view and choose the plot type you wish to see. There is also an option to include or remove the moving average for the plot and at what interval you wish to view the moving average. For plots such as Renko and PNF displaying the moving average may not be helpful.

In [5]:
tickers = list(df_dict)

@ipyw.interact(
Mavg = ipyw.widgets.Dropdown(options=[('5Day', 5), ('10Day', 10), ('30Day', 30)],description='Moving_Average:'),
MA = True,
Tickers = tickers,
Plot_Type = ['ohlc','line','hollow_and_filled','renko','pnf']
)

def PlotIt(Tickers,Plot_Type,MA,Mavg):
    if MA == True:
      Mavg
      mpf.plot(df_dict[Tickers],type = Plot_Type,volume = True,mav = Mavg,figratio = (100,40),style = 'yahoo',title = (Tickers + ' Plot With Moving Average'))
    else:
      mpf.plot(df_dict[Tickers],type = Plot_Type,volume = True,figratio = (100,40),style = 'yahoo',title = Tickers);

interactive(children=(Dropdown(description='Tickers', options=('SOL1-USD', 'BTC-USD', 'ETH-USD'), value='SOL1-…

# Compare to Each other

In [6]:
# New Dataframe with Close for each ticker
close = pd.DataFrame()

for a in df_dict:
    cl = pd.DataFrame(df_dict[a],columns=['Close'])
    cl = cl.rename(columns={'Close': a }).reset_index(level=0).set_index('Date')
    close = pd.concat([close,cl],axis = 1)

close = close.reset_index(level = 0)
close

,Date,SOL1-USD,BTC-USD,ETH-USD
0,2020-11-11,2.002125,15701.339844,462.960541
1,2020-11-12,1.899138,16276.343750,461.005280
2,2020-11-13,2.020792,16317.808594,474.626434
3,2020-11-14,1.917016,16068.138672,460.149841
4,2020-11-15,2.043234,15955.587891,447.559082
...,...,...,...,...
361,2021-11-07,249.823486,63326.988281,4620.554688
362,2021-11-08,248.467178,67566.828125,4812.087402
363,2021-11-09,239.213135,66971.828125,4735.068848
364,2021-11-10,233.779526,64995.230469,4636.174316
